<a href="https://colab.research.google.com/github/nelslindahlx/NLP/blob/master/Yet_another_working_GPT2_corpus_example_with_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This was working code and the corpus file was live on 6/7/2020.

> Please note that synthetic writing could be mean or otherwise terrible... my apologies in advance for what is generated from the corpus... 



# Working example of GPT-2 using gpt-2-simple

The best method to use the OpenAI GPT-2 model is via gpt-2-simple. It works well enough. Some methods that I tried and that failed are commented out before it. The only method that seems to work is the gpt-2-simple

In [0]:
# !pip uninstall tensorflow -y 
# !pip uninstall tensorflow-gpu -y
# !pip install tensorflow==1.14
# !pip install tensorflow-gpu==1.4.1
# !pip3 install tensorflow==1.12.0
!pip install gpt-2-simple

You can now use the 1x TensorFlow selection and check what GPU you have. 

In [0]:
%tensorflow_version 1.x
!nvidia-smi

This is our import of the GPT-2 we installed above. You get the warning that is the root of the GPT-2 problem noting that "The TensorFlow contrib module will not be included in TensorFlow 2.0.'

*   List item
*   List item



In [0]:
import gpt_2_simple as gpt2
import os
import requests


This snippet is using the 124M model. The other model numbers are commented out above for later adventure. I tried the 774M model and that failed.  

In [0]:
# !python3 download_model.py 124M
# !python3 download_model.py 355M
# !python3 download_model.py 774M
# !python3 download_model.py 1558M
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /content/models


This snippet is using the 355M model. 

In [0]:
# !python3 download_model.py 124M
# !python3 download_model.py 355M
# !python3 download_model.py 774M
# !python3 download_model.py 1558M
model_name = "355M"
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /content/models


# Getting the corpus file

To be able to go get files we need gdown

In [0]:
pip install gdown

We need to download the corpus file to do some training on it... This one gets the corpus I plan on using from a shareable Google Drive link

In [0]:
!gdown --id 1S7euXz9txzbrLTXJKl7IY5p6rd3jr0qn --output nlindahl.txt

Set the file name...

In [0]:
file_name = "nlindahl.txt"

Let's run the 355M model. This will take about 20 minutes to run... 

*   This does have 2 warnings right now for tf.where in 2.0 and a deprication warning for get_backward_walk_ops
*   sample_every was increased to 100 vs. 200. This makes it easier to watch. it has no other beneift



In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=100,
              save_every=500
              )

Or instead of the 355M you could run the 124M model. This will take about *20* minutes to run... 

*   This does have 2 warnings right now for tf.where in 2.0 and a deprication warning for get_backward_walk_ops
*   sample_every was increased to 100 vs. 200

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run2',
              print_every=10,
              sample_every=100,
              save_every=500
              )

# Let's make some outputs

This prompt will generate text from the trained model

In [0]:
gpt2.generate(sess, run_name='run1')

You can change the prefix text, but it spews nonsense so far...

In [0]:
single_text = gpt2.generate(sess, prefix="What makes Phil happy")
print(single_text)

Another generate with a prefix

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Today I wrote about",
              nsamples=5,
              batch_size=5
              )

# Messing around with doing a zip of the models

Doing one shot trainign worked well enough above. I'm now going to try to train this with more shots and save the checkpoint

In [0]:
!zip -r /content/models/file.zip /content/models/355M